In [1]:
import matplotlib.pyplot as plt
import scipy as sp
import numpy as np
import os
import pandas as pd

from skimage import morphology, measure
from skimage.segmentation import expand_labels, watershed, relabel_sequential
from sklearn.preprocessing import minmax_scale
from scipy import ndimage as ndi
from dask_ml.cluster import KMeans
from dask_ml.decomposition import PCA

from dask_image import imread
from dask_image import ndmeasure
from dask_image import ndfilters
from dask import array as da
from dask import delayed
from dask.distributed import Client, progress
from colorsys import hls_to_rgb

import napari

In [2]:
client = Client(processes=False, threads_per_worker=6,
                n_workers=1, memory_limit='12GB')
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://172.19.111.216:8787/status,
Dashboard: http://172.19.111.216:8787/status,Workers: 1
Total threads: 6,Total memory: 11.18 GiB
Status: running,Using processes: False
Comm: inproc://172.19.111.216/12888/1,Workers: 0
Dashboard: http://172.19.111.216:8787/status,Total threads: 0
Started: Just now,Total memory: 0 B
Comm: inproc://172.19.111.216/12888/4,Total threads: 6
Dashboard: http://172.19.111.216:52252/status,Memory: 11.18 GiB
Nanny: None,


In [69]:
#Specific is the name of the folder of the orginal data
specific = 'F231006L'
#which_finger should be 1,2,3, or 4, corresponding to the index, middle, ring, and pinky fingers
finger_num = 1
digit_num = 2

Adjust the numbers below based on what you see in Napari

In [70]:
filter_size = 4
closing_size = 13
threshold = 375

In [71]:
finger_names = {1: 'index', 2: 'middle', 3: 'ring', 4: 'pinky'}
finger_name = finger_names[finger_num]
digit_names = {1: 'proximal', 2: 'middle', 3: 'distal'}
digit_name = digit_names[digit_num]

In [72]:
full_finger_dir = f'../fingers/{finger_name}/full/{specific}/'
digit_dir = f'../fingers/{finger_name}/{digit_name}/{specific}/'
finger_eig_dir = f'../fingers/{finger_name}/full/{specific} - eigens/'
mesh_dir = f'../meshes/{finger_name}/{digit_name}/{specific}/'
mesh_csv_dir = f'../mesh_csvs/{finger_name}/{digit_name}/{specific}/'
digit_eig_dir = f'../fingers/{finger_name}/{digit_name}/{specific} - eigens/'
spacing = np.load('../spacing/' + specific + '.npy')

In [73]:
all_dirs = [digit_dir, mesh_dir, mesh_csv_dir]
for d in all_dirs:
    os.makedirs(d, exist_ok=True)

In [74]:
digit = np.pad(
    da.from_npy_stack(digit_dir).compute(),
    ((closing_size,closing_size),)*3,
    'edge',
)

In [75]:
viewer = napari.Viewer()

viewer.add_image(
    digit,
    scale = spacing,
)

<Image layer 'digit' at 0x1a3c6fd4050>

In [76]:
smoothed = ndi.median_filter(
    digit,
    size = filter_size
)

In [77]:
viewer.add_image(
    smoothed,
    scale = spacing,
)

<Image layer 'smoothed' at 0x1a3c6fd4b90>

In [78]:
thresholded = morphology.remove_small_objects(
    smoothed > threshold, min_size = 20**3
)

In [79]:
closed = morphology.closing(
    thresholded,
    footprint = morphology.ball(
        closing_size,
        decomposition = 'sequence',
    ),
    mode = 'ignore',
)

In [80]:
viewer.add_image(
    closed,
    scale = spacing,
)

<Image layer 'closed' at 0x1a3c7253110>

In [81]:
holes = measure.label(
    closed == 0
)

In [82]:
viewer.add_labels(
    holes,
    scale = spacing,
)

<Labels layer 'holes' at 0x1a3fa9be5d0>

In [83]:
n_holes = np.max(holes) - 1
fill_holes = np.array([1] + [0] + [1]*n_holes, dtype=np.dtype('uint8'))
solid = fill_holes[holes]

In [84]:
viewer.add_image(
    solid,
    scale = spacing,
    contrast_limits = (0,1),
)

<Image layer 'solid' at 0x1a3f8efd590>

In [85]:
verts, faces, normals, values = measure.marching_cubes(
    volume = solid,
    spacing = spacing,
)

In [86]:
mesh_view = viewer.add_surface(
    data = (verts, faces, values),
)

In [87]:
np.save(mesh_dir + 'vertices.npy', verts)
np.save(mesh_dir + 'faces.npy', faces)
np.save(mesh_dir + 'values.npy', values)
np.save(mesh_dir + 'normals.npy', normals)

In [88]:
np.savetxt(mesh_csv_dir + 'vertices.csv', verts, delimiter=',')
np.savetxt(mesh_csv_dir + 'faces.csv', faces, delimiter=',')
np.savetxt(mesh_csv_dir + 'values.csv', values, delimiter=',')
np.savetxt(mesh_csv_dir + 'normals.csv', normals, delimiter=',')